Maclay Teefey (mjt6vj)
Data Project #1 Midterm

# DS-2002 – Data Project 1 100 points


The goal of this project is to demonstrate (1) an understanding of and (2) competence creating and 
implementing basic data science systems such as pipelines, scripts, data transformations, APIs, databases 
and cloud services. Submit your project in your GitHub Repo or file drop on Collab. 
Data Projects must be done individually.

## ETL Data Processor

You project should demonstrate your understanding of the differing types of data systems (OLTP/OLAP), 
and how data can be extracted from various source systems (structured, semi-structured, unstructured), 
transformed (cleansed, integrated), and then loaded into a destination system that’s optimized for post 
hoc diagnostic analysis.

# Deliverable

## 1. Design a dimensional data mart that represents a simple business process of your choosing.

a. Examples might include retail sales, inventory management, procurement, order 
management, transportation or hospitality bookings, medical appointments, student 
registration and/or attendance.

b. You may select any business process that interests you, but remember that a 
dimensional data mart provides for the post hoc summarization and historic analysis of 
business transactions that reflect the interaction between various entities (e.g., patients 
& doctors, retailers & customers, students & schools/classes, travelers & airlines/hotels).

In [173]:
import os
import cryptography
import numpy
import pandas as pd
from sqlalchemy import create_engine

In [174]:
host_name = "localhost"
host_ip = "127.0.0.1"
port = "3306"
user_id = "root"
pwd = "m6zIUutf0k1$"

src_dbname = "world"
dst_dbname = "data_project_warehouse"

In [175]:
conn_str = f"mysql+pymysql://{user_id}:{pwd}@{host_name}"
sqlEngine = create_engine(conn_str, pool_recycle=3600)

sqlEngine.execute(f"DROP DATABASE IF EXISTS `{dst_dbname}`;")
sqlEngine.execute(f"CREATE DATABASE `{dst_dbname}`;")
sqlEngine.execute(f"USE {dst_dbname};")

The three data sets I will be using and their forms:

1. World sample data set from Oracle in SQL

2. World Happiness Index 2019 data set downloaded as a CSV file from Kaggle (local file system) (link: https://www.kaggle.com/datasets/unsdsn/world-happiness)

3. Nobel Prize Laureates from the Nobel Prizes API (taken in as JSON)

The relationship being warehoused is between the countries and the nobel prize winners. The key joining point will be the country name and birth country of the laureate. The required date portion will be the Birth Date section.

## 2. Develop an ETL pipeline that extracts, transforms, and loads data into your data mart.

a. Extract data from one or more SQL database tables; hosted locally or in the Cloud.

b. Retrieve a data file, either from a remote or local file system, converting its original 
format (e.g., CSV, JSON) into a SQL database table.

c. Modify the number of columns from each source to the destination.

d. Provide error messages wherever an operation fails (i.e., Try/Except error handlers).


Getting the Countries table from World Data

Functions for Getting Data From and Setting Data Into Databases Taken from Lab 03

In [177]:
def get_dataframe(user_id, pwd, host_name, db_name, sql_query):
    conn_str = f"mysql+pymysql://{user_id}:{pwd}@{host_name}/{db_name}"
    sqlEngine = create_engine(conn_str, pool_recycle=3600)
    connection = sqlEngine.connect()
    dframe = pd.read_sql(sql_query, connection);
    connection.close()
    
    return dframe


def set_dataframe(user_id, pwd, host_name, db_name, df, table_name, pk_column, db_operation):
    conn_str = f"mysql+pymysql://{user_id}:{pwd}@{host_name}/{db_name}"
    sqlEngine = create_engine(conn_str, pool_recycle=3600)
    connection = sqlEngine.connect()
    
    if db_operation == "insert":
        df.to_sql(table_name, con=connection, index=False, if_exists='replace')
        sqlEngine.execute(f"ALTER TABLE {table_name} ADD PRIMARY KEY ({pk_column});")
            
    elif db_operation == "update":
        df.to_sql(table_name, con=connection, index=False, if_exists='append')
    
    connection.close()

In [178]:
sql_world = "SELECT * FROM world.country;"
df_world = get_dataframe(user_id, pwd, host_name, src_dbname, sql_world)
df_world.head(2)

,Code,Name,Continent,Region,SurfaceArea,IndepYear,Population,LifeExpectancy,GNP,GNPOld,LocalName,GovernmentForm,HeadOfState,Capital,Code2
0,ABW,Aruba,North America,Caribbean,193.0,NaN,103000,78.4,828.0,793.0,Aruba,Nonmetropolitan Territory of The Netherlands,Beatrix,129.0,AW
1,AFG,Afghanistan,Asia,Southern and Central Asia,652090.0,1919.0,22720000,45.9,5976.0,NaN,Afganistan/Afqanestan,Islamic Emirate,Mohammad Omar,1.0,AF


Now to get World Happiness data

In [179]:
df_happiness = pd.read_csv("happiness-2019.csv")
df_happiness.head(2)

,Overall rank,Country or region,Score,GDP per capita,Social support,Healthy life expectancy,Freedom to make life choices,Generosity,Perceptions of corruption
0,1,Finland,7.769,1.340,1.587,0.986,0.596,0.153,0.393
1,2,Denmark,7.600,1.383,1.573,0.996,0.592,0.252,0.410


And finally the nobel prize data

In [180]:
# to handle  data retrieval
import urllib3
from urllib3 import request

import numpy as np
# to handle certificate verification
import certifi

# to manage json data
import json

# I am using https://plainenglish.io/blog/from-api-to-pandas-getting-json-data-with-python-df127f699b6b 
# as a guide to convert from JSON to a pandas dataframe

In [181]:
# handle certificate verification and SSL warnings
# https://urllib3.readthedocs.io/en/latest/user-guide.html#ssl

http = urllib3.PoolManager(
       cert_reqs='CERT_REQUIRED',
       ca_certs=certifi.where())

In [182]:
# get data from the API
url = 'https://api.nobelprize.org/2.1/laureates'

r = http.request('GET', url)
status_check = r.status
status_check # I will be using this as one of the "try catch" without it being a try catch

200

In [183]:
if status_check == 200:
    # decode json data into a dict object
    data = json.loads(r.data.decode('utf-8'))
else:
    print("Error access API")

I would print out the full data but it is too big to print out as a full chunk

In [184]:
# in this dataset, the data to extract is under 'laureates'
df_nobel = pd.json_normalize(data, 'laureates')
df_nobel.head(2)

,id,fileName,gender,sameAs,links,nobelPrizes,knownName.en,knownName.se,givenName.en,givenName.se,...,death.place.countryNow.en,death.place.countryNow.no,death.place.countryNow.se,death.place.countryNow.sameAs,death.place.continent.en,death.place.continent.no,death.place.continent.se,death.place.locationString.en,death.place.locationString.no,death.place.locationString.se
0,745,spence,male,"[https://www.wikidata.org/wiki/Q157245, https:...","[{'rel': 'laureate', 'href': 'https://api.nobe...","[{'awardYear': '2001', 'category': {'en': 'Eco...",A. Michael Spence,A. Michael Spence,A. Michael,A. Michael,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,102,bohr,male,"[https://www.wikidata.org/wiki/Q103854, https:...","[{'rel': 'laureate', 'href': 'https://api.nobe...","[{'awardYear': '1975', 'category': {'en': 'Phy...",Aage N. Bohr,Aage N. Bohr,Aage N.,Aage N.,...,Denmark,Danmark,Danmark,[https://www.wikidata.org/wiki/Q35],Europe,Europa,Europa,"Copenhagen, Denmark","København, Danmark","Köpenhamn, Danmark"


The key that is needed to unite the data sets is in this

In [185]:
df_world["Name"].head(2)

0          Aruba
1    Afghanistan
Name: Name, dtype: object

In [186]:
df_happiness["Country or region"].head(2)

0    Finland
1    Denmark
Name: Country or region, dtype: object

In [187]:
df_nobel["birth.place.country.en"].head(2)

0        USA
1    Denmark
Name: birth.place.country.en, dtype: object

There is a clear issue with each dataframe having inconsistent naming schemes

I will now get the important details needed from each table and rename the categories into their data mart names.

In [188]:
df_world_trimmed = df_world[["Name", "Code", "Population", "GNP", "LifeExpectancy"]].copy()
df_world_trimmed.reset_index(inplace=True)
df_world_trimmed.rename(columns={"Name":"Country", "Code":"CountryCode", "index":"CountryKey"}, inplace=True)
df_world_trimmed.head(2)

,CountryKey,Country,CountryCode,Population,GNP,LifeExpectancy
0,0,Aruba,ABW,103000,828.0,78.4
1,1,Afghanistan,AFG,22720000,5976.0,45.9


In [189]:
df_happiness_trimmed = df_happiness[["Country or region", "Score"]].copy()
df_happiness_trimmed.reset_index(inplace=True)
df_happiness_trimmed.rename(columns={"Country or region":"Country", "Score":"HappinessScore", "index":"HappinessKey"}, inplace=True)
df_happiness_trimmed.head(2)

,HappinessKey,Country,HappinessScore
0,0,Finland,7.769
1,1,Denmark,7.600


In [190]:
df_nobel.columns

Index(['id', 'fileName', 'gender', 'sameAs', 'links', 'nobelPrizes',
       'knownName.en', 'knownName.se', 'givenName.en', 'givenName.se',
       'familyName.en', 'familyName.se', 'fullName.en', 'fullName.se',
       'birth.date', 'birth.place.city.en', 'birth.place.city.no',
       'birth.place.city.se', 'birth.place.country.en',
       'birth.place.country.no', 'birth.place.country.se',
       'birth.place.cityNow.en', 'birth.place.cityNow.no',
       'birth.place.cityNow.se', 'birth.place.cityNow.sameAs',
       'birth.place.countryNow.en', 'birth.place.countryNow.no',
       'birth.place.countryNow.se', 'birth.place.countryNow.sameAs',
       'birth.place.continent.en', 'birth.place.continent.no',
       'birth.place.continent.se', 'birth.place.locationString.en',
       'birth.place.locationString.no', 'birth.place.locationString.se',
       'wikipedia.slug', 'wikipedia.english', 'wikidata.id', 'wikidata.url',
       'death.date', 'death.place.city.en', 'death.place.city.no',
   

In [191]:
df_nobel_trimmed = df_nobel[["fullName.en","birth.date", "birth.place.locationString.en","birth.place.country.en", "birth.place.continent.en"]].copy()
df_nobel_trimmed.reset_index(inplace=True)
df_nobel_trimmed.rename(columns={"index":"NobelKey","fullName.en":"LaureateName", "birth.date":"BirthDate","birth.place.locationString.en":"BirthLocation", "birth.place.country.en":"Country", "birth.place.continent.en":"BirthContinent"}, inplace=True)
df_nobel_trimmed.loc[df_nobel_trimmed["Country"] == "USA", "Country"] = "United States"
df_nobel_trimmed.head(2)

,NobelKey,LaureateName,BirthDate,BirthLocation,Country,BirthContinent
0,0,A. Michael Spence,1943-00-00,"Montclair, NJ, USA",United States,North America
1,1,Aage Niels Bohr,1922-06-19,"Copenhagen, Denmark",Denmark,Europe


I will create smaller fact tables for nobel and country

In [192]:
df_fact_country = pd.merge(df_world_trimmed,df_happiness_trimmed, how="inner", left_on="Country", right_on="Country")
df_fact_country.head()

,CountryKey,Country,CountryCode,Population,GNP,LifeExpectancy,HappinessKey,HappinessScore
0,1,Afghanistan,AFG,22720000,5976.0,45.9,153,3.203
1,4,Albania,ALB,3401200,3205.0,71.6,106,4.719
2,7,United Arab Emirates,ARE,2441000,37966.0,74.1,20,6.825
3,8,Argentina,ARG,37032000,340238.0,75.1,46,6.086
4,9,Armenia,ARM,3520000,1813.0,66.4,115,4.559


And then I will create a date table based upon the birthDate category

In [193]:
min_date = df_nobel_trimmed["BirthDate"].min()
max_date = df_nobel_trimmed["BirthDate"].max()
print(str(min_date) + "-" + str(max_date))

1835-10-31-1976-08-15


In [194]:
# Here is the code borrowed from https://stackoverflow.com/questions/47150709/how-to-create-a-calendar-table-date-dimension-in-pandas
# to create a 
def create_date_table(start='2000-01-01', end='2050-12-31'):
        df = pd.DataFrame({"Date": pd.date_range(start, end)})
        df["Day"] = df.Date.dt.day_name()
        df["Week"] = df.Date.dt.weekofyear
        df["Quarter"] = df.Date.dt.quarter
        df["Year"] = df.Date.dt.year
        df["Year_half"] = (df.Quarter + 1) // 2
        return df

In [204]:
df_dim_date = create_date_table(min_date, max_date)
df_dim_date.reset_index(inplace=True)
df_dim_date.rename(columns={"index":"DateKey"}, inplace=True)
df_dim_date.head()

C:\Users\student\AppData\Local\Temp\ipykernel_17068\632955866.py:6: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated. Please use Series.dt.isocalendar().week instead.
  df["Week"] = df.Date.dt.weekofyear


,DateKey,Date,Day,Week,Quarter,Year,Year_half
0,0,1835-10-31,Saturday,44,4,1835,2
1,1,1835-11-01,Sunday,44,4,1835,2
2,2,1835-11-02,Monday,45,4,1835,2
3,3,1835-11-03,Tuesday,45,4,1835,2
4,4,1835-11-04,Wednesday,45,4,1835,2


In [205]:
df_dim_date_string = df_dim_date.copy().astype('string')
df_fact_nobel = pd.merge(df_nobel_trimmed,df_dim_date_string, how="inner", left_on="BirthDate", right_on="Date")
df_fact_nobel = df_fact_nobel.drop(["Date"],axis=1)
df_fact_nobel.head()

,NobelKey,LaureateName,BirthDate,BirthLocation,Country,BirthContinent,DateKey,Day,Week,Quarter,Year,Year_half
0,1,Aage Niels Bohr,1922-06-19,"Copenhagen, Denmark",Denmark,Europe,31642,Monday,25,2,1922,1
1,2,Aaron Ciechanover,1947-10-01,"Haifa, British Protectorate of Palestine (now ...",British Protectorate of Palestine,Asia,40877,Wednesday,40,4,1947,2
2,3,Aaron Klug,1926-08-11,"Zelvas, Lithuania",Lithuania,Europe,33156,Wednesday,32,3,1926,2
3,5,Abdus Salam,1926-01-29,"Jhang Maghiāna, India (now Pakistan)",India,Asia,32962,Friday,4,1,1926,1
4,6,Abhijit Banerjee,1961-02-21,"Mumbai, India",India,Asia,45769,Tuesday,8,1,1961,1


In [202]:
df_fact_combined = pd.merge(df_fact_nobel,df_fact_country, how="inner", left_on="Country", right_on="Country")
df_fact_combined.reset_index(inplace=True)
df_fact_combined = df_fact_combined.drop(["Quarter","Population"], axis=1) # Will be used in the select statements
df_fact_combined.rename(columns={"index":"id"}, inplace=True)
df_fact_combined.head()

,id,NobelKey,LaureateName,BirthDate,BirthLocation,Country,BirthContinent,DateKey,Day,Week,Year,Year_half,CountryKey,CountryCode,GNP,LifeExpectancy,HappinessKey,HappinessScore
0,0,1,Aage Niels Bohr,1922-06-19,"Copenhagen, Denmark",Denmark,Europe,31642,Monday,25,1922,1,59,DNK,174099.0,76.5,1,7.600
1,1,3,Aaron Klug,1926-08-11,"Zelvas, Lithuania",Lithuania,Europe,33156,Wednesday,32,1926,2,126,LTU,10692.0,69.1,41,6.149
2,2,5,Abdus Salam,1926-01-29,"Jhang Maghiāna, India (now Pakistan)",India,Asia,32962,Friday,4,1926,1,99,IND,447114.0,62.5,139,4.015
3,3,6,Abhijit Banerjee,1961-02-21,"Mumbai, India",India,Asia,45769,Tuesday,8,1961,1,99,IND,447114.0,62.5,139,4.015
4,4,7,Abiy Ahmed Ali,1976-08-15,"Beshasha, Ethiopia",Ethiopia,Africa,51423,Sunday,33,1976,2,68,ETH,6353.0,45.2,133,4.286


In [227]:
df_fact_combined.shape

(18, 18)

Now to add all of the data to the data warehouse

In [203]:
db_operation = "insert"

tables = [('fact_combined', df_fact_combined, 'id'), ('world', df_world_trimmed, 'CountryKey'), ('happiness', df_happiness_trimmed, 'HappinessKey'), ('nobel', df_nobel_trimmed, 'NobelKey'), ('dim_date', df_dim_date, 'DateKey')]

for table_name, dataframe, primary_key in tables:
    set_dataframe(user_id, pwd, host_name, dst_dbname, dataframe, table_name, primary_key, db_operation)

## 3. Author one or more SQL queries (SELECT statements) to demonstrate proper functionality.

a. SELECT data from at least 3 tables (two dimensions; plus the fact table).

b. Perform some type of aggregation (e.g., SUM, COUNT, AVERAGE). This, of course, 
necessitates some form of grouping operation (e.g., GROUP BY <customer.last_name>).


In [237]:
sql_select_statement = """
SELECT `fact_combined`.Country as country_name,
       `world`.Population as population,
       COUNT(*) as count_of_laureates,
       `nobel`.BirthContinent as continent
FROM `data_project_warehouse`.`fact_combined`
INNER JOIN `data_project_warehouse`.`world`
ON  `fact_combined`.`CountryKey` = `world`.`CountryKey`
INNER JOIN `data_project_warehouse`.`nobel`
ON  `fact_combined`.`NobelKey` = `nobel`.`NobelKey`
GROUP BY `world`.`country`
ORDER BY count_of_laureates DESC;
"""
df_result_one = get_dataframe(user_id, pwd, host_name, src_dbname, sql_select_statement)
df_result_one.head()

,country_name,population,count_of_laureates,continent
0,United States,278357000,3,North America
1,India,1013662000,2,Asia
2,Germany,82164700,2,Europe
3,Japan,126714000,2,Asia
4,Denmark,5330000,1,Europe


The 3 Tables used in this select statment are:

1. fact_combined with the country name
2. world with the country population
3. nobel with the birth continent of the nobel laurette

The low numbers are because the api only contains 25 entries so after the inner joins 18 laurettes are left